In [264]:
import pandas as pd
from sklearn.utils import shuffle
import plotly.express as px

pd.options.mode.chained_assignment = None  # default='warn'

dataset = pd.read_csv('msd_genre_dataset0.csv', header=0)

classicPopAndRock = dataset.iloc[0:23895]
punk = dataset.iloc[23895:27095]
folk = dataset.iloc[27095:40287]
pop = dataset.iloc[40287:41904]
danceAndElectronica = dataset.iloc[41904:46839]
metal = dataset.iloc[46839:48942]
jazz = dataset.iloc[48942:53276]
classical = dataset.iloc[53276:55150]
hipHop = dataset.iloc[55150:55584]
soulAndReggae = dataset.iloc[55584:59600]

classicPopAndRock.name = 'classicPopAndRock'
punk.name = 'punk'
folk.name = 'folk'
pop.name = 'pop'
danceAndElectronica.name = 'danceAndElectronica'
jazz.name = 'jazz'
classical.name = 'classical'
hipHop.name = 'hipHop'
soulAndReggae.name = 'soulAndReggae'

def extractTimbreMeans(df):
    avgTimbreDf = df[['avg_timbre1','avg_timbre2','avg_timbre3','avg_timbre4',
                      'avg_timbre5','avg_timbre6','avg_timbre7','avg_timbre8',
                      'avg_timbre9','avg_timbre10','avg_timbre11','avg_timbre12']]
    df['avgTimbreMean'] = avgTimbreDf.sum(axis=1)
    varTimbreDf = df[['var_timbre1','var_timbre2','var_timbre3','var_timbre4',
                      'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
                      'var_timbre9','var_timbre10','var_timbre11','var_timbre12']]
    df['varTimbreMean'] = varTimbreDf.sum(axis=1)
    return df

def extractVarTimbre(df):
    varTimbreDf = df[['var_timbre1','var_timbre2','var_timbre3','var_timbre4',
                      'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
                      'var_timbre9','var_timbre10','var_timbre11','var_timbre12']]
    df['mean'] = varTimbreDf.mean(axis=1)
    return df

def randomSampling(df):
    return df.head(10)

# classicPopAndRockAvgTimbre = extractAvgTimbre(classicPopAndRock)
# punkAvgTimbre = extractAvgTimbre(punk)
# folkAvgTimbre = extractAvgTimbre(folk)
# popAvgTimbre = extractAvgTimbre(classicPopAndRock)
# danceAndElectronicaAvgTimbre = extractAvgTimbre(danceAndElectronica)
# metalAvgTimbre = extractAvgTimbre(metal)
# jazzAvgTimbre = extractAvgTimbre(jazz)
# classicalAvgTimbre = extractAvgTimbre(classical)
# hipHopAvgTimbre = extractAvgTimbre(hipHop)
# soulAndReggaeAvgTimbre = extractAvgTimbre(soulAndReggae)

# classicPopAndRockVarTimbre = extractAvgTimbre(classicPopAndRock)
# punkVarTimbre = extractVarTimbre(punk)
# folkVarTimbre = extractVarTimbre(folk)
# popVarTimbre = extractVarTimbre(classicPopAndRock)
# danceAndElectronicaVarTimbre = extractVarTimbre(danceAndElectronica)
# metalVarTimbre = extractVarTimbre(metal)
# jazzVarTimbre = extractVarTimbre(jazz)
# classicalVarTimbre = extractVarTimbre(classical)
# hipHopVarTimbre = extractVarTimbre(hipHop)
# soulAndReggaeVarTimbre = extractVarTimbre(soulAndReggae)

datasetFrames = [randomSampling(classicPopAndRock),randomSampling(punk),
                   randomSampling(folk),randomSampling(pop),
                   randomSampling(danceAndElectronica),randomSampling(metal),
                   randomSampling(jazz),randomSampling(classical),randomSampling(hipHop),
                   randomSampling(soulAndReggae)]
datasetDataframe = pd.concat(datasetFrames)
datasetSample = shuffle(datasetDataframe)
avgTimbreSample = extractTimbreMeans(datasetSample)
avgTimbre1 = 'avg_timbre9'
avgTimbre2 = 'avg_timbre10'
#avgTimbreOneSample = avgTimbreSample[['genre',avgTimbre]]

fig = px.scatter(avgTimbreSample, x=avgTimbre1, y=avgTimbre2, color='genre')
fig.show()

# fig = px.box(avgTimbreSample, x="genre", y=avgTimbre1)
# fig.show()